<a href="https://colab.research.google.com/github/ritzdevp/Tweet-Author-Classification/blob/main/models/EDL_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd

In [4]:
!git clone https://github.com/ritzdevp/Tweet-Author-Classification.git

fatal: destination path 'Tweet-Author-Classification' already exists and is not an empty directory.


In [5]:
data = pd.read_csv('/content/Tweet-Author-Classification/cleaned_tweets.csv', index_col=[0])

In [6]:
len(data)

38230

In [7]:
authors2remove = ['elonmusk', 'tyler', 'joebiden', 'openai', 'jeffbezos', 'dhh','sundarpichai', 'ivankatrump']

In [8]:
data.columns = ['tweet', 'author']

In [9]:
data.head()

,tweet,author
0,🤣,elonmusk
1,♥ ️ 🚀 ✨,elonmusk
2,spacex falcon team making great progress aimin...,elonmusk
3,thank sir,elonmusk
4,🤣 🤣,elonmusk


In [10]:
data = data[data["author"].str.contains("elonmusk|tyler|joebiden|openai|jeffbezos|dhh|sundarpichai|ivankatrump|narendramodi") == False]

In [11]:
data = data.reset_index(drop=True)

In [12]:
data.head()

,tweet,author
0,wishing celebrate today blessed joyful easter ...,barackobama
1,michelle send warmest wish gathering tonight p...,barackobama
2,mayor washington meant lot michelle — like man...,barackobama
3,traveled world together he's captured iconic s...,barackobama
4,75th anniversary jackie robinson day we're rem...,barackobama


In [13]:
len(data)

31286

In [14]:
data['author'] = pd.factorize(data['author'])[0]

In [15]:
data.head()

,tweet,author
0,wishing celebrate today blessed joyful easter ...,0
1,michelle send warmest wish gathering tonight p...,0
2,mayor washington meant lot michelle — like man...,0
3,traveled world together he's captured iconic s...,0
4,75th anniversary jackie robinson day we're rem...,0


In [16]:
data = data.dropna()

In [17]:
!pip install -U sentence-transformers

In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [19]:
sentences = data['tweet']

In [20]:
len(list(sentences))

30939

In [21]:
sentences = data['tweet']
embeddings = model.encode(list(sentences))

In [22]:
embeddings.shape

(30939, 384)

In [23]:
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding.shape)
    break

Sentence: wishing celebrate today blessed joyful easter difficult couple year let give thanks gift enjoy — people make life special
Embedding: (384,)


In [24]:
bert_features = np.array(embeddings)

In [25]:
np.save('bert_features.npy', bert_features)

In [26]:
X = bert_features.copy()

In [27]:
y = list(data['author']).copy()

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
len(X_train[0])

384

In [30]:
num_classes = len(set(y_train))
print(num_classes)

13


In [31]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, learning_rate_init=0.001, learning_rate='adaptive',
                    hidden_layer_sizes=(384
                                        , 256, 64, 13), random_state=1, verbose=False, max_iter=100)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.5990088343029519


In [32]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, learning_rate_init=0.001, learning_rate='adaptive',
                    hidden_layer_sizes=(384
                                        , 400, 256, 64, 13), random_state=1, verbose=False, max_iter=100)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.5874811463046757


In [33]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='adam', alpha=1e-5, learning_rate_init=0.01, learning_rate='adaptive',
                    hidden_layer_sizes=(384
                                        ,128, 64, 13), random_state=1, verbose=False, max_iter=100)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.5753070458952811


# SVM

In [34]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
svm_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svm_clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [35]:
pred = svm_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.6412411118293472


# RANDOM FOREST

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(max_depth=10, random_state=0)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=0)

In [37]:
pred = rf_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.48642533936651583


#LOGISTIC REGRESSION


In [38]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(random_state=0, max_iter=100).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [39]:
pred = lr_clf.predict(X_test)
print(accuracy_score(y_test, pred))

0.6025641025641025
